In [3]:
import pandas as pd

# 사용자 프로파일 파일 이름
profile_file = "user_profile.csv"

# 사용자 프로파일 입력/업데이트 함수
def initialize_user_profile():
    try:
        # 기존 프로파일 불러오기
        user_profile = pd.read_csv(profile_file).iloc[0].to_dict()
        print("기존 사용자 프로파일이 불러와졌습니다:")
        print(user_profile)
    except FileNotFoundError:
        # 새로운 프로파일 입력 받기
        print("사용자 프로파일을 새로 입력합니다.")
        user_profile = input_user_profile()

    # 사용자에게 업데이트 여부를 물어보기
    update = input("프로파일을 업데이트하시겠습니까? (네/아니오): ")
    if update.lower() == "네":
        user_profile = update_user_profile(user_profile)

    # 최신화된 프로파일 저장
    pd.DataFrame([user_profile]).to_csv(profile_file, index=False)
    print("최신화된 사용자 프로파일이 저장되었습니다.")
    return user_profile

# 사용자 프로파일 입력 함수
def input_user_profile():
    name = input("이름을 입력하세요: ")
    age = int(input("나이를 입력하세요: "))
    gender = input("성별을 입력하세요 (남/여): ")
    weight = float(input("체중 (kg)을 입력하세요: "))
    height = float(input("키 (cm)을 입력하세요: "))
    goal = input("운동 목표를 입력하세요 (예: 근력 증가, 체중 감량, 유지): ")

    return {
        "name": name,
        "age": age,
        "gender": gender,
        "weight": weight,
        "height": height,
        "goal": goal
    }

# 사용자 프로파일 업데이트 함수
def update_user_profile(user_profile):
    print("프로파일 업데이트를 시작합니다. 변경하고 싶은 항목만 입력하고, 그대로 유지할 항목은 Enter를 누르세요.")

    name = input(f"이름 [{user_profile['name']}]: ")
    if name:
        user_profile["name"] = name

    age = input(f"나이 [{user_profile['age']}]: ")
    if age:
        user_profile["age"] = int(age)

    gender = input(f"성별 [{user_profile['gender']}]: ")
    if gender:
        user_profile["gender"] = gender

    weight = input(f"체중 (kg) [{user_profile['weight']}]: ")
    if weight:
        user_profile["weight"] = float(weight)

    height = input(f"키 (cm) [{user_profile['height']}]: ")
    if height:
        user_profile["height"] = float(height)

    goal = input(f"운동 목표 [{user_profile['goal']}]: ")
    if goal:
        user_profile["goal"] = goal

    print("프로파일이 업데이트되었습니다.")
    print(user_profile)
    return user_profile

# 사용자 프로파일 불러오기
user_profile = initialize_user_profile()
print("최신화된 사용자 프로파일:")
print(user_profile)


기존 사용자 프로파일이 불러와졌습니다:
{'name': '이석환', 'age': 23, 'gender': '남', 'weight': 69.0, 'height': 175.0, 'goal': '근력 증가'}


프로파일 업데이트를 시작합니다. 변경하고 싶은 항목만 입력하고, 그대로 유지할 항목은 Enter를 누르세요.
프로파일이 업데이트되었습니다.
{'name': '이석환', 'age': 23, 'gender': '남', 'weight': 70.0, 'height': 175.0, 'goal': '근력 증가'}
최신화된 사용자 프로파일이 저장되었습니다.
최신화된 사용자 프로파일:
{'name': '이석환', 'age': 23, 'gender': '남', 'weight': 70.0, 'height': 175.0, 'goal': '근력 증가'}


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# 한글 폰트 설정 (운영체제에 맞게 변경)   -> 그래프 그릴 때 한글이 깨져서 추가함.
plt.rcParams["font.family"] = "Malgun Gothic"  # Windows
# plt.rcParams["font.family"] = "AppleGothic"  # macOS
# plt.rcParams["font.family"] = "NanumGothic"  # Ubuntu
plt.rcParams["axes.unicode_minus"] = False

# Ollama 모델 설정
llama_model = ChatOllama(model="llama3.1")
chat_context = []

# 운동 기록 파일 이름
record_file = "workout_log.csv"

# 사용자 프로파일 파일 이름
profile_file = "user_profile.csv"

# CSV 파일 초기화 함수
def initialize_csv():
    try:
        pd.read_csv(record_file)
        print("운동 기록 파일이 이미 존재합니다.")
    except FileNotFoundError:                     # 운동 기록 파일이 없는 경우 판다스를 활용하여 csv 파일로 저장하여 파일 생성
        columns = ["date", "exercise", "set_number", "weight", "reps"]
        pd.DataFrame(columns=columns).to_csv(record_file, index=False)
        print("운동 기록 파일이 생성되었습니다.")

# 세트별 운동 기록 입력 함수   - 날짜, 운동이름, 세트, 무게, 횟수 입력받아 운동 기록 파일에 추가
def log_workout():
    date = input("운동 날짜를 입력하세요 (YYYY-MM-DD) [오늘 날짜로 자동 입력하려면 Enter]: ")
    if not date:
        date = datetime.now().strftime("%Y-%m-%d")
        print(f"날짜가 입력되지 않아, 오늘 날짜로 기록합니다: {date}")
    exercise = input("운동 이름을 입력하세요: ")
    set_count = int(input("몇 세트를 수행하셨나요? "))

    records = []
    for set_number in range(1, set_count + 1):
        weight = float(input(f"{set_number}세트의 무게 (kg): "))
        reps = int(input(f"{set_number}세트의 반복 횟수: "))
        records.append({"date": date, "exercise": exercise, "set_number": set_number, "weight": weight, "reps": reps})

    df = pd.read_csv(record_file)  #
    df = pd.concat([df, pd.DataFrame(records)], ignore_index=True) 
    df.to_csv(record_file, index=False)
    print("운동 기록이 저장되었습니다!")

def get_latest_exercise_data(exercise_name):
    try:
        df = pd.read_csv(record_file)
    except FileNotFoundError:
        return "운동 기록이 없습니다."

    # 특정 운동의 최신 기록만 추출
    exercise_data = df[df["exercise"].str.contains(exercise_name, case=False)].tail(5)
    if exercise_data.empty:
        return f"{exercise_name}에 대한 기록이 없습니다."

    # 최신 기록을 텍스트로 변환
    return exercise_data.to_string(index=False)

def query_ollama(user_input):
    global chat_context

    # 그래프 요청 판단
    if "그래프" in user_input or "보여줘" in user_input:
        show_graph(user_input)
        return "그래프를 표시합니다."

    # 최신 운동 기록 불러오기 (벤치프레스 예시)
    if "벤치프레스" in user_input:
        latest_data = get_latest_exercise_data("벤치프레스")
    else:
        latest_data = "운동 기록이 없습니다."

    # 대화 메시지 추가
    chat_context.append(HumanMessage(content=user_input))

    # Ollama 모델에게 운동 기록과 함께 요청
    chat_prompt = [
        SystemMessage(content=f"""
        너는 한국어로 대답하는 헬스 트레이너야. 
        사용자의 운동 기록은 다음과 같아:\n\n{latest_data}\n\n
        사용자의 프로파일은 다음과 같아: {profile_file}
        이 기록을 참고해서 피드백과 운동 루틴을 추천해줘. 
        새로운 정보를 추측하지 말고, 제공된 기록만 바탕으로 조언해줘.
        """),
        *chat_context
    ]
    response = llama_model(chat_prompt)
    chat_context.append(AIMessage(content=response.content))

    return response.content


# 서브플롯 그래프 생성 함수
def plot_exercise_graph(data, exercise_name, set_number):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

    # 첫 번째 그래프 (무게)
    ax1.plot(data["date"], data["weight"], 'bo-', label="무게 (kg)")
    ax1.set_title(f"{exercise_name} - Set {set_number}")
    ax1.set_xlabel("날짜")
    ax1.set_ylabel("무게 (kg)")
    ax1.grid()
    ax1.legend(loc='upper left', frameon=False)
    for i, value in enumerate(data["weight"]):
        ax1.text(data["date"].iloc[i], value, f"{value:.1f} kg", ha='center', va='bottom', fontsize=10)

    # 두 번째 그래프 (반복 횟수)
    ax2.plot(data["date"], data["reps"], 'r--x', label="반복 횟수")
    ax2.set_xlabel("날짜")
    ax2.set_ylabel("반복 횟수")
    ax2.grid()
    ax2.legend(loc='upper left', frameon=False)
    for i, value in enumerate(data["reps"]):
        ax2.text(data["date"].iloc[i], value, f"{value}회", ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.show()

# 운동 기록 그래프 표시 함수
def show_graph(user_input):
    try:
        exercise_name = user_input.split()[0]
        df = pd.read_csv(record_file)
    except FileNotFoundError:
        print("운동 기록이 없습니다.")
        return

    exercise_data = df[df["exercise"].str.contains(exercise_name, case=False)]
    if exercise_data.empty:
        print(f"{exercise_name} 기록이 없습니다.")
        return

    for set_number in exercise_data["set_number"].unique():
        set_data = exercise_data[exercise_data["set_number"] == set_number]
        plot_exercise_graph(set_data, exercise_name, set_number)

# 사용자와의 대화 함수
def chat_with_user():
    print("헬스 트레이너 챗봇과 대화를 시작합니다.")
    print("그래프를 보고 싶다면, 운동 이름을 맨 앞에 두고 요청해 주세요.")
    print("예: '스쿼트 기록을 그래프로 보여줘' 또는 '벤치 프레스 그래프 보여줘'")
    print("'종료'라고 입력하면 대화를 마칩니다.")

    while True:
        user_input = input("사용자: ")
        if user_input.lower() == "종료":
            print("대화를 종료합니다.")
            break

        response = query_ollama(user_input)
        print(f"챗봇: {response}")

# 메인 실행 함수
def main():
    initialize_csv()
    while True:
        print("\n1. 운동 기록 추가")
        print("2. 헬스 트레이너와 대화하기")
        print("3. 종료")
        choice = input("선택하세요 (1/2/3): ")

        if choice == "1":
            log_workout()
        elif choice == "2":
            chat_with_user()
        elif choice == "3":
            print("프로그램을 종료합니다.")
            break
        else:
            print("잘못된 입력입니다. 다시 선택해 주세요.")

if __name__ == "__main__":
    main()

운동 기록 파일이 이미 존재합니다.

1. 운동 기록 추가
2. 헬스 트레이너와 대화하기
3. 종료
헬스 트레이너 챗봇과 대화를 시작합니다.
그래프를 보고 싶다면, 운동 이름을 맨 앞에 두고 요청해 주세요.
예: '스쿼트 기록을 그래프로 보여줘' 또는 '벤치 프레스 그래프 보여줘'
'종료'라고 입력하면 대화를 마칩니다.
챗봇: 사용자의 벤치프레스 기록을 분석하면 다음과 같다:

* 2024-11-15: 4세트, 70kg, 10회
* 2024-11-16: 1~3세트, 80kg, 7~5회 (무게 감소)
* 2024-11-16: 4세트, 70kg, 10회 (무게 다시 원복)

이 기록을 기준으로, 사용자가 벤치프레스 할 때는 약간의 무게 감소를 보여주고, 마지막 세트에서는 원래의 무게로 돌아오기 때문에, 다음번에 할 때는 이전과 같은 무게 (약 70kg)를 추천한다.
대화를 종료합니다.

1. 운동 기록 추가
2. 헬스 트레이너와 대화하기
3. 종료
프로그램을 종료합니다.
